## Vi testar vår model på en video stream
Vi börjar med att installera yt_dlp med pip

In [1]:
%pip install yt_dlp

Note: you may need to restart the kernel to use updated packages.


### Sedan kör vi på yt video

In [10]:
import cv2 as cv
import time
from yt_dlp import YoutubeDL
from ultralytics import YOLO

best_strea_url = None

# Levi Main
video_url = 'https://www.youtube.com/watch?v=Wr9b5aYA4mI'

model = YOLO('output/yolo_model_new/weights/best.pt')

# Get the best stream url
ydl_opts = {
    "format": "best",
}

with YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(video_url, download=False)
    best_strea_url = info['url']
    
# Create a new VideoCapture object and read the video from the best stream url
cap = cv.VideoCapture(best_strea_url)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Class colors for bounding boxes
class_colors = {
    'skier': (255,0,0),
    'snowboarder': (0,0,255),
}

# Define counting line position (adjust as needed)
y_line = 780

# Track object positions
object_tracks = {}  # Tracks last known y positions of objects
last_count_time = {}  # Stores timestamp of last count
COOLDOWN_TIME = 2  # Cooldown in seconds before recounting

# Counters
counters = {
    'skier': 0,
    'snowboarder': 0,
}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    results = model(frame, device="cuda:0")
    
    current_time = time.time()
    
    for result in results:
        for i, box in enumerate(result.boxes):
            print(box)

            cls = box.cls
            label = model.names[int(cls)]
            confidence = box.conf.item()
            color = class_colors.get(label, (255,255,255))
            
            # Get bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            center_y = (y1 + y2) // 2

            # Unique object ID (based on detection index)
            obj_id = f"{label}_{i}"
            
            # Check if object crosses the counting line
            if obj_id in object_tracks:
                prev_y = object_tracks[obj_id]
                
                # Check if object has crossed the line
                if prev_y < y_line <= center_y:
                    last_time = last_count_time.get(obj_id, 0)
                    
                    if (current_time - last_time) > COOLDOWN_TIME:
                        if label == 'skier':
                            counters['skier'] += 1
                        elif label == 'snowboarder':
                            counters['snowboarder'] += 1
                            
                        last_count_time[obj_id] = current_time

            object_tracks[obj_id] = center_y
            
            # Draw bounding box
            frame = cv.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            # Draw label and count
            label_text = f"{label} {counters[label]}"
            cv.putText(frame, label_text, (x1, y2 + 5), cv.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    
    # Draw counting line
    cv.line(frame, (0, y_line), (frame.shape[1], y_line), (0, 255, 0), 3)
    
    # Render the frame
    cv.imshow("Frame", frame)
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.waitKey(1)
cv.destroyAllWindows()

[youtube] Extracting URL: https://www.youtube.com/watch?v=Wr9b5aYA4mI
[youtube] Wr9b5aYA4mI: Downloading webpage
[youtube] Wr9b5aYA4mI: Downloading tv client config
[youtube] Wr9b5aYA4mI: Downloading player 6b3caec8
[youtube] Wr9b5aYA4mI: Downloading tv player API JSON
[youtube] Wr9b5aYA4mI: Downloading ios player API JSON
[youtube] Wr9b5aYA4mI: Downloading m3u8 information
[youtube] Wr9b5aYA4mI: Downloading m3u8 information

0: 384x640 (no detections), 12.0ms
Speed: 1.6ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.4ms
Speed: 1.9ms preprocess, 11.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.7ms
Speed: 2.2ms preprocess, 11.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.6ms
Speed: 1.9ms preprocess, 11.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.9ms
Spe